In [5]:
import tarfile
import os
import numpy as np
import pickle
import torch
from preprocess._utils import *
smooth_method = 'fft'
transform = MinMaxScaler(feature_range=(-1, 1))

In [6]:

# extract .tar.gz file
source_path = os.getcwd()
filename = "exported_data"
tf = tarfile.open(os.path.join(source_path, filename + ".tar.gz"))
tf.extractall()
print("extract finish")

extract finish


In [60]:
import math
def str_to_float(str_num):
    '''
    Change textual scientific notation into a floating-point number
    '''
    before_e = float(str_num.split('e')[0])
    sign = str_num.split('e')[1][:1]
    after_e = int(str_num.split('e')[1][1:])

    if sign == '+':
        float_num = before_e * math.pow(10, after_e)
    elif sign == '-':
        float_num = before_e * math.pow(10, -after_e)
    else:
        float_num = None
        print('error: unknown sign')
    return float_num

# calculate the number of worms
files = os.listdir("./" + filename)
num = int(len(files) / 6) # every worm has 6 txt files
data_dict = {}

for i in range(0, 1):
    worm = "worm" + str(i)

    real_data = []
    with open(filename + "/" + str(i) + "_gcamp.txt", "r") as f:
        for line in f.readlines():
            cal = list(map(float, line.split(" ")))
            real_data.append(cal)
    real_data = np.array(real_data) # format: (time, neuron)

    sc = transform  # normalize data
    real_data = sc.fit_transform(real_data)

    delta = (real_data[:, 83] - real_data[:, 82])
    
    print(f'mean:{delta.mean()}\nshape:{delta.shape}\nmin:{delta.min()}\nmax:{delta.max()}')
    
    label_list = []
    with open("./" + filename + "/" + str(i) + "_labels.txt", "r") as f:
        for line in f.readlines():
            l = line.strip("\n")
            label_list.append(l)

   
    for j, val in enumerate(label_list):
        if val == '' or val == 'smthng else':
            label_list[j] = str(j+ len(label_list))

    neuron_to_idx = {}
    neurons_copy = []
    for neuron in label_list:
        if neuron not in set(neurons_copy) and label_list.count(neuron) > 1:
            neurons_copy.append(neuron + "L")
    
    print(neurons_copy)
    


    print("---", len(label_list), real_data.shape)
    print("------", len(neuron_to_idx))
    print(label_list, neuron_to_idx)


    num_named = len(neuron_to_idx)

    sc = transform  # normalize data
    real_data = sc.fit_transform(real_data)
    real_data = torch.tensor(
            real_data, dtype=torch.float32
        )  # add a feature dimension and convert to tensor
    smooth_real_data, residual, smooth_residual = smooth_data_preprocess(
        real_data, smooth_method
    )
    
    timeVectorSeconds = []
    with open("./" + filename + "/" + str(i) + "_t.txt", "r") as f:
        for line in f.readlines():
            l = line.strip("\n")
            timeVectorSeconds.append(str_to_float(l))
    
    time_in_seconds = np.array(timeVectorSeconds)
    time_in_seconds = torch.tensor(time_in_seconds).to(torch.float32)
    dt = torch.zeros_like(time_in_seconds).to(torch.float32)
    dt[1:] = time_in_seconds[1:] - time_in_seconds[:-1]

    num_neurons = real_data.shape[1]
    max_time = real_data.shape[0]

    print(
            "len. Ca recording %s, total num. neurons %s, num. ID'd neurons %s"
            % (max_time, num_neurons, num_named),
            end="\n\n",
    )

    data_dict.update(
            {
                worm: {
                    "dataset": "Leifer2023",
                    "worm": worm,
                    "calcium_data": real_data,
                    "smooth_calcium_data": smooth_real_data,
                    "residual_calcium": residual,
                    "smooth_residual_calcium": smooth_residual,
                    "neuron_to_idx": neuron_to_idx,
                    "idx_to_neuron": dict((v, k) for k, v in neuron_to_idx.items()),
                    "max_time": int(max_time),
                    "time_in_seconds": time_in_seconds,
                    "dt": dt,
                    "num_neurons": int(num_neurons),
                    "num_named_neurons": num_named,
                    "num_unknown_neurons": int(num_neurons) - num_named,
                },
            }
        )
    
    print(len(data_dict[worm]["neuron_to_idx"]), len(data_dict[worm]["idx_to_neuron"]))
    
    # standardize the shape of calcium data to 302 x time
    data_dict[worm] = reshape_calcium_data(data_dict[worm])
    
    # # pickle the data
    # file = os.path.join(processed_path, "Leifer2023.pickle")
    # pickle_out = open(file, "wb")
    # pickle.dump(data_dict, pickle_out)
    # pickle_out.close()
    # pickle_in = open(file, "rb")
    # Leifer2023 = pickle.load(pickle_in)
    # print(Leifer2023.keys(), end="\n\n")


#     cal_noNull = []
#     l_noNull = []
#     # subtract data without label
#     for j in range(0, len(label_list)):
#         if label_list[j] != "":
#             cal_noNull.append(real_data[j])
#             l_noNull.append(label_list[j])

#     cal_noNull_array = np.array(cal_noNull)
#     gcamp_np.append(dict(zip(l_noNull, cal_noNull_array)))
# print("------data reformatted------")
# print(str(num) + " txt have been reformatted.")

# file = open("gcamp.pickle", "wb")
# pickle.dump(gcamp_np, file)
# file.close()
# print("------save to Pickle------")

mean:-0.10157551800984738
shape:(4454,)
min:-1.1795625039931958
max:1.3235919496305641
['ASELL', 'ASELL', 'ASELL', 'SAADL', 'ASGRL', 'ASGRL', 'SAADL', 'FLPRL', 'AWBRL', '48L', '48L', 'RIGL', 'RIGL', 'FLPRL', 'AWBRL']
--- 114 (4454, 114)
------ 0
['114', 'IL2V', 'BAGL', 'ASEL', 'ASEL', 'SMDVL', 'AMsoL', 'I1L', '122', '123', 'RMED', '125', 'NSML', 'M3L', 'M1', 'RMDVL', 'ASEL', 'AVDL', 'AWBL', 'AVEL', 'FLPL', '135', '136', '137', '138', '139', '140', 'I3', 'URXL', '143', '144', 'AVAL', '146', '147', 'SMDDL', 'AIYL', 'AIZL', 'AMsoR', 'OLQDR', '153', '154', 'IL1VL', '156', 'SAAD', '158', 'AQR', 'RMDDL', '161', '162', '163', 'AVJR', 'AWAR', '166', '167', 'I1R', 'OLLR', 'AVDR', 'ASGR', 'M3R', 'ASGR', 'OLQVR', '175', 'SAAD', '177', '178', '179', 'DB1', '181', 'VA1', '183', '184', '185', 'I2R', '187', '188', 'FLPR', '190', 'AWBR', '41', '193', 'AIZR', '48', '48', 'DB2', 'VB1', 'RIG', 'RIG', '201', '202', '203', '204', '205', '206', 'FLPR', '208', 'AWBR', '21', 'AVER', '212', '213', '214', '215'

AssertionError: Number of neurons in calcium dataset does not match number of recorded neurons.